# Fuentes

### Link: https://stats.stackexchange.com/questions/352036/what-should-i-do-when-my-neural-network-doesnt-learn
Sin palabras.

### Link: https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/
Explicación sobre las redes neuronales con múltiples capas, y lo que más interesante me pareció fue la enumeración de diferentes enfoques para saber cómo proponer la cantidad de capas y las neurones por cada capa a utilizar en el modelo de la red neuronal.

### Link: https://medium.com/fintechexplained/what-are-hidden-layers-4f54f7328263
En este artículo hace una explicación de las capas de una red neuronal de múltiples capas, útil para diferencias capa de entrada, salida y capas ocultas o intermedias.

### Link: https://mmuratarat.github.io/2019-06-12/embeddings-with-numeric-variables-Keras
Este artículo es importante porque te explica cómo usar la API funcional de Keras, que fue útil a la hora de utilizar embeddings porque necesitabamos que las nuevas entradas del modelo generadas por la capa de embedding se juntaran con las demás variables del problema, para lo cual tuvimos que emplear una capa *concatenate*, que funciona al utilizar la API funciona de Keras.

### Link: https://stackoverflow.com/questions/61367382/plot-custom-data-with-tensorboard
Esta discusión de StackOverflow muestra un código de muestra de Python para registrar información personalizada usando TensorBoard, me resulto de gran utilidad para poder concentrar toda la información del proceso de entrenamiento, validación y evaluación de una red neuronal, cada vez que probaba, en un mismo lugar.

### Link: https://branyang.gitbooks.io/tfdocs/content/get_started/embedding_viz.html#metadata
Este link me sirvió para entender un poco más las opciones de visualización de embedding en TensorBoard, y para poder configurar el archivo de metadata para visualizar bien dónde quedaba cada uno de los valores de la variable o feature luego de la capa de embeddig.

### Link: https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/
Este artículo me sirvió para leer un poco más sobre funciones de activación, cómo se suelen usar en modelos con múltiples capas, algunos detalles me parecieron interesantes como el hecho de no cambiar la función activación a lo largo de las capas de un modelo (lo que usualmente se hace en la práctica según explican), y luego el hecho de que en las redes MLP suele predominar el uso de la **RELU**, aunque no entran en detalle con respecto a sus variantes pero sí dicen que se pueden utilizar. Esto me da la pauta de que debería mantener en las capas ocultas una **RELU** o alguna de sus variantes, pero no sigmoide o tangente hiperbólica porque pueden presentar un problema de **vanishing gradient**.

### Link: https://mlfromscratch.com/activation-functions-explained/#/
Otro artículo interesante que explica las funciones de activación, sus ventajas y desventajas, entre otras cuestiones. Es útil para tener más o menos una idea de qué problemas puede causar cada función de activación, me resultó útil para darme cuenta de algunas cosas que podían estar afectando al modelo.

# 1. Cargando base de datos

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
import matplotlib.pyplot as plt

In [3]:
import numpy as np

In [4]:
import importlib

In [5]:
import sys

In [6]:
sys.path.insert(0, '..')

In [7]:
sys.path.insert(0, '../..')

In [8]:
# Read the database from the .csv file into a pandas dataframe
df = pd.read_csv('../../databases/insurance.csv')

# 2. Preprocesamiento de los datos

In [9]:
from sklearn import preprocessing

## 2.1. Codificación de variables no numéricas o categóricas

In [10]:
# Create a label encoder for the sex variable or feature and create a new column in the dataframe 
# with the encoded version of the gender
sex_encoder = preprocessing.LabelEncoder()
sex_encoder.fit(df['sex'])
df['sex-encoded'] = sex_encoder.transform(df['sex'])

In [11]:
# Create a label encoder for the smoker variable or feature and create a new column in the dataframe
# with the encoded version of the smoker
smoker_encoder = preprocessing.LabelEncoder()
smoker_encoder.fit(df['smoker'])
df['smoker-encoded'] = smoker_encoder.transform(df['smoker'])

In [12]:
# Create a label encoder for the region variable or feature and create a new column in the dataframe
# with the encoded version of the region
region_encoder = preprocessing.LabelEncoder()
region_encoder.fit(df['region'])
df['region-encoded'] = region_encoder.transform(df['region'])

In [13]:
df.head()

,age,sex,bmi,children,smoker,region,charges,sex-encoded,smoker-encoded,region-encoded
0,19,female,27.900,0,yes,southwest,16884.92400,0,1,3
1,18,male,33.770,1,no,southeast,1725.55230,1,0,2
2,28,male,33.000,3,no,southeast,4449.46200,1,0,2
3,33,male,22.705,0,no,northwest,21984.47061,1,0,1
4,32,male,28.880,0,no,northwest,3866.85520,1,0,1


## 2.2. Filtrado de variables

In [14]:
# Filtering or removing of non desired variables
df_x = df[['age', 'bmi', 'smoker-encoded', 'children', 'sex-encoded', 'region-encoded']]
df_y = df['charges']

# 3. Separación del conjunto de entrenamiento y evaluación

In [19]:
from sklearn import model_selection

## 3.1. Separación de los conjuntos
Es importante notar que, se realiza la separación del conjunto de datos original en **train**, **valid** y **test**, por fuera del framework de Keras para garantizar un adecuado tratamiento de los conjuntos acorde a la metodología empleada. En otras palabras, de esta forma nos aseguramos que cualquier preprocesamiento o normalización sobre validación (valid) y evaluación (test) se realiza a partir de la información obtenida en entrenamiento.

In [30]:
# Split the dataset into train_valid and test
x_train_valid, x_test, y_train_valid, y_test = model_selection.train_test_split(df_x, df_y, test_size=0.2, random_state=1, shuffle=True)

In [31]:
# Split the dataset into train and valid
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(x_train_valid, y_train_valid, test_size=0.2, random_state=1, shuffle=True)

## 3.2. Normalización de variables

In [32]:
# Select the variables where the z-score will be applied
scalable_variables = ['bmi', 'age']

if scalable_variables:
    # Create an instance of the StandardScaler for each variable
    scaler = preprocessing.StandardScaler()

    # Fit the distribution
    scaler.fit(x_train.loc[:, scalable_variables])

    # Transform and normalize all variables
    x_train.loc[:, scalable_variables] = scaler.transform(x_train.loc[:, scalable_variables])
    x_valid.loc[:, scalable_variables] = scaler.transform(x_valid.loc[:, scalable_variables])
    x_test.loc[:, scalable_variables] = scaler.transform(x_test.loc[:, scalable_variables])

## 3.3. Conjuntos de entrenamiento, validación y evaluación
Particularmente, para las redes neuronales de múltiples capas, en este problema vamos a estar utilizando una capa de *embedding* para poder reducir la dimensionalidad necesaria para desacoplar las categorías de la variable *region*. Es decir, normalmente podríamos utilizar un *one hot encoding* para independizar en diferentes variables, pero produce una mayor dimensionalidad y poca interpretabilidad de lo que la red neuronal aprende durante el entrenamiento. En conclusión, debemos separar los grupos de entradas por el formato de entrada de la red neuronal.

In [33]:
x_train = [x_train[['age', 'bmi', 'smoker-encoded', 'children', 'sex-encoded']], x_train['region-encoded']]
x_valid = [x_valid[['age', 'bmi', 'smoker-encoded', 'children', 'sex-encoded']], x_valid['region-encoded']]
x_test = [x_test[['age', 'bmi', 'smoker-encoded', 'children', 'sex-encoded']], x_test['region-encoded']]

# 4. Multilayer Perceptron

In [57]:
from src import mlp_helper
importlib.reload(mlp_helper);

In [35]:
mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                           hidden_layers=1,
                           units_per_layer=6,
                           hidden_layer_activation='elu',
                           epochs=1000,
                           batch_size=32,
                           learning_rate=1,
                           decay_rate=0.01,
                           optimizer='adam',
                           beta_1=0.99,
                           beta_2=0.999
                          )

Model logs at tb-logs/mlp/20210524-192433
Model checkpoints at checkpoints/mlp/20210524-192433
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 2)         8           input_4[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 2)            0           embedding_1[0][0]              

In [36]:
mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                           hidden_layers=2,
                           units_per_layer=6,
                           hidden_layer_activation='elu',
                           epochs=200,
                           batch_size=32,
                           learning_rate=1, 
                           decay_rate=0.01,
                           optimizer='adam',
                           beta_1=0.99,
                           beta_2=0.999
                          )

Model logs at tb-logs/mlp/20210524-192739
Model checkpoints at checkpoints/mlp/20210524-192739
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 2)         8           input_6[0][0]                    
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_2 (Flatten)             (None, 2)            0           embedding_2[0][0]              

In [37]:
mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                           hidden_layers=2,
                           units_per_layer=6,
                           hidden_layer_activation='relu',
                           epochs=200,
                           batch_size=32,
                           learning_rate=1, 
                           decay_rate=0.01,
                           optimizer='adam',
                           beta_1=0.99,
                           beta_2=0.999
                          )

Model logs at tb-logs/mlp/20210524-192845
Model checkpoints at checkpoints/mlp/20210524-192845
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 1, 2)         8           input_8[0][0]                    
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_3 (Flatten)             (None, 2)            0           embedding_3[0][0]              

In [39]:
mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                           hidden_layers=3,
                           units_per_layer=6,
                           hidden_layer_activation='elu',
                           epochs=500,
                           batch_size=32,
                           learning_rate=0.8, 
                           decay_rate=0.05,
                           optimizer='adam',
                           beta_1=0.99,
                           beta_2=0.999
                          )

Model logs at tb-logs/mlp/20210524-193145
Model checkpoints at checkpoints/mlp/20210524-193145
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 1, 2)         8           input_12[0][0]                   
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_5 (Flatten)             (None, 2)            0           embedding_5[0][0]              

In [38]:
mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                           hidden_layers=1,
                           units_per_layer=150,
                           hidden_layer_activation='relu',
                           epochs=1000,
                           batch_size=32,
                           learning_rate=2, 
                           decay_rate=0.01,
                           optimizer='adam',
                           beta_1=0.99,
                           beta_2=0.999
                          )

Model logs at tb-logs/mlp/20210524-193025
Model checkpoints at checkpoints/mlp/20210524-193025
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 1, 2)         8           input_10[0][0]                   
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_4 (Flatten)             (None, 2)            0           embedding_4[0][0]              

In [40]:
mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                           hidden_layers=1,
                           units_per_layer=150,
                           hidden_layer_activation='elu',
                           epochs=1000,
                           batch_size=32,
                           learning_rate=2, 
                           decay_rate=0.01,
                           optimizer='adam',
                           beta_1=0.99,
                           beta_2=0.999
                          )

Model logs at tb-logs/mlp/20210524-193232
Model checkpoints at checkpoints/mlp/20210524-193232
Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 1, 2)         8           input_14[0][0]                   
__________________________________________________________________________________________________
input_13 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_6 (Flatten)             (None, 2)            0           embedding_6[0][0]              

In [41]:
mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                           hidden_layers=3,
                           units_per_layer=30,
                           hidden_layer_activation='relu',
                           epochs=200,
                           batch_size=64,
                           learning_rate=0.5, 
                           decay_rate=0.01,
                           optimizer='adam',
                           beta_1=0.99,
                           beta_2=0.999
                          )

Model logs at tb-logs/mlp/20210524-193448
Model checkpoints at checkpoints/mlp/20210524-193448
Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 1, 2)         8           input_16[0][0]                   
__________________________________________________________________________________________________
input_15 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_7 (Flatten)             (None, 2)            0           embedding_7[0][0]              

In [42]:
mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                           hidden_layers=4,
                           units_per_layer=30,
                           hidden_layer_activation='relu',
                           epochs=200,
                           batch_size=64,
                           learning_rate=0.5, 
                           decay_rate=0.01,
                           optimizer='adam',
                           beta_1=0.99,
                           beta_2=0.999
                          )

Model logs at tb-logs/mlp/20210524-193558
Model checkpoints at checkpoints/mlp/20210524-193558
Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 1, 2)         8           input_18[0][0]                   
__________________________________________________________________________________________________
input_17 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_8 (Flatten)             (None, 2)            0           embedding_8[0][0]              

In [43]:
mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                           hidden_layers=5,
                           units_per_layer=30,
                           hidden_layer_activation='relu',
                           epochs=200,
                           batch_size=64,
                           learning_rate=0.5, 
                           decay_rate=0.01,
                           optimizer='adam',
                           beta_1=0.99,
                           beta_2=0.999
                          )

Model logs at tb-logs/mlp/20210524-193759
Model checkpoints at checkpoints/mlp/20210524-193759
Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 1, 2)         8           input_20[0][0]                   
__________________________________________________________________________________________________
input_19 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_9 (Flatten)             (None, 2)            0           embedding_9[0][0]              

In [44]:
mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                           hidden_layers=5,
                           units_per_layer=30,
                           hidden_layer_activation='elu',
                           epochs=200,
                           batch_size=64,
                           learning_rate=0.5, 
                           decay_rate=0.01,
                           optimizer='adam',
                           beta_1=0.99,
                           beta_2=0.999
                          )

Model logs at tb-logs/mlp/20210524-193938
Model checkpoints at checkpoints/mlp/20210524-193938
Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 1, 2)         8           input_22[0][0]                   
__________________________________________________________________________________________________
input_21 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_10 (Flatten)            (None, 2)            0           embedding_10[0][0]            

In [45]:
mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                           hidden_layers=2,
                           units_per_layer=150,
                           hidden_layer_activation='elu',
                           epochs=200,
                           batch_size=64,
                           learning_rate=0.5, 
                           decay_rate=0.01,
                           optimizer='adam',
                           beta_1=0.99,
                           beta_2=0.999
                          )

Model logs at tb-logs/mlp/20210524-194025
Model checkpoints at checkpoints/mlp/20210524-194025
Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, 1, 2)         8           input_24[0][0]                   
__________________________________________________________________________________________________
input_23 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_11 (Flatten)            (None, 2)            0           embedding_11[0][0]            

In [53]:
mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                           hidden_layers=2,
                           units_per_layer=150,
                           hidden_layer_activation='relu',
                           epochs=500,
                           batch_size=64,
                           learning_rate=0.5, 
                           decay_rate=0.01,
                           optimizer='adam',
                           beta_1=0.99,
                           beta_2=0.999
                          )

Model logs at tb-logs/mlp/20210524-195649
Model checkpoints at checkpoints/mlp/20210524-195649
Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_40 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_19 (Embedding)        (None, 1, 2)         8           input_40[0][0]                   
__________________________________________________________________________________________________
input_39 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_19 (Flatten)            (None, 2)            0           embedding_19[0][0]            

In [61]:
mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                           hidden_layers=2,
                           units_per_layer=150,
                           hidden_layer_activation='relu',
                           epochs=500,
                           batch_size=64,
                           learning_rate=0.5, 
                           decay_rate=0.01,
                           optimizer='adam',
                           beta_1=0.99,
                           beta_2=0.999,
                           regularizer='l2',
                           regularizer_lambda=1e-4
                          )

Model logs at tb-logs/mlp/20210524-203444
Model checkpoints at checkpoints/mlp/20210524-203444
Model: "model_24"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_52 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_25 (Embedding)        (None, 1, 2)         8           input_52[0][0]                   
__________________________________________________________________________________________________
input_51 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_25 (Flatten)            (None, 2)            0           embedding_25[0][0]            

In [56]:
mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                           hidden_layers=2,
                           units_per_layer=150,
                           hidden_layer_activation='relu',
                           use_batch_normalization=True,
                           epochs=500,
                           batch_size=64,
                           learning_rate=0.5, 
                           decay_rate=0.01,
                           optimizer='adam',
                           beta_1=0.99,
                           beta_2=0.999
                          )

Model logs at tb-logs/mlp/20210524-201153
Model checkpoints at checkpoints/mlp/20210524-201153
Model: "model_21"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_46 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_22 (Embedding)        (None, 1, 2)         8           input_46[0][0]                   
__________________________________________________________________________________________________
input_45 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_22 (Flatten)            (None, 2)            0           embedding_22[0][0]            

In [58]:
mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                           hidden_layers=3,
                           units_per_layer=150,
                           hidden_layer_activation='elu',
                           epochs=200,
                           batch_size=64,
                           learning_rate=0.5, 
                           decay_rate=0.01,
                           optimizer='adam',
                           beta_1=0.99,
                           beta_2=0.999
                          )

Model logs at tb-logs/mlp/20210524-202354
Model checkpoints at checkpoints/mlp/20210524-202354
Model: "model_22"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_48 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_23 (Embedding)        (None, 1, 2)         8           input_48[0][0]                   
__________________________________________________________________________________________________
input_47 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_23 (Flatten)            (None, 2)            0           embedding_23[0][0]            

In [54]:
mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                           hidden_layers=3,
                           units_per_layer=150,
                           hidden_layer_activation='elu',
                           use_batch_normalization=True,
                           epochs=200,
                           batch_size=64,
                           learning_rate=0.5, 
                           decay_rate=0.01,
                           optimizer='adam',
                           beta_1=0.99,
                           beta_2=0.999
                          )

Model logs at tb-logs/mlp/20210524-200323
Model checkpoints at checkpoints/mlp/20210524-200323
Model: "model_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_42 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_20 (Embedding)        (None, 1, 2)         8           input_42[0][0]                   
__________________________________________________________________________________________________
input_41 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_20 (Flatten)            (None, 2)            0           embedding_20[0][0]            

In [47]:
mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                           hidden_layers=1,
                           units_per_layer=1500,
                           hidden_layer_activation='elu',
                           epochs=200,
                           batch_size=64,
                           learning_rate=0.5, 
                           decay_rate=0.01,
                           optimizer='adam',
                           beta_1=0.99,
                           beta_2=0.999
                          )

Model logs at tb-logs/mlp/20210524-194351
Model checkpoints at checkpoints/mlp/20210524-194351
Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_28 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 1, 2)         8           input_28[0][0]                   
__________________________________________________________________________________________________
input_27 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_13 (Flatten)            (None, 2)            0           embedding_13[0][0]            

In [50]:
mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                           hidden_layers=1,
                           units_per_layer=1500,
                           hidden_layer_activation='tanh',
                           epochs=200,
                           batch_size=64,
                           learning_rate=0.5, 
                           decay_rate=0.01,
                           optimizer='adam',
                           beta_1=0.99,
                           beta_2=0.999
                          )

Model logs at tb-logs/mlp/20210524-195005
Model checkpoints at checkpoints/mlp/20210524-195005
Model: "model_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_34 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_16 (Embedding)        (None, 1, 2)         8           input_34[0][0]                   
__________________________________________________________________________________________________
input_33 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_16 (Flatten)            (None, 2)            0           embedding_16[0][0]            

In [51]:
mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                           hidden_layers=1,
                           units_per_layer=1500,
                           hidden_layer_activation='sigmoid',
                           epochs=200,
                           batch_size=64,
                           learning_rate=0.5, 
                           decay_rate=0.01,
                           optimizer='adam',
                           beta_1=0.99,
                           beta_2=0.999
                          )

Model logs at tb-logs/mlp/20210524-195207
Model checkpoints at checkpoints/mlp/20210524-195207
Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_36 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_17 (Embedding)        (None, 1, 2)         8           input_36[0][0]                   
__________________________________________________________________________________________________
input_35 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_17 (Flatten)            (None, 2)            0           embedding_17[0][0]            

In [59]:
mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                           hidden_layers=2,
                           units_per_layer=600,
                           hidden_layer_activation='relu',
                           use_batch_normalization=True,
                           epochs=500,
                           batch_size=64,
                           learning_rate=1.0, 
                           decay_rate=0.01,
                           optimizer='adam',
                           beta_1=0.99,
                           beta_2=0.999
                          )

Model logs at tb-logs/mlp/20210524-202743
Model checkpoints at checkpoints/mlp/20210524-202743
Model: "model_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_50 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_24 (Embedding)        (None, 1, 2)         8           input_50[0][0]                   
__________________________________________________________________________________________________
input_49 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_24 (Flatten)            (None, 2)            0           embedding_24[0][0]            